In [1]:
import baostock as bs
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import pymongo
import json
from datetime import datetime, date
from Kline import DayKline,WeekKline,MonthKline
from IPython.core.debugger import set_trace

bsLoggedIn = False
def customLogin():
    global bsLoggedIn
    if not bsLoggedIn:
        bs.login()
        bsLoggedIn = True
def customLogout():
    global bsLoggedIn
    if bsLoggedIn:
        bs.logout()
        bsLoggedIn = False
rsiOverBuy = 80
rsiOverSell = 20
rsiMiddle = 50

深发展


In [6]:
# dataList = []
# dayk = DayKline("sz.000001", "深发展")
# dayK2 = DayKline("sz.000002", "万科A")
# dataList.append(dayk.__dict__)
# dataList.append(dayK2.__dict__)
# dataListJson = json.dumps(dataList, ensure_ascii=False) 
# mydb = connectDB()
# collection = mydb["Kline_Day"]
# collection.insert_many(dataList)
customLogin()
rs = bs.query_stock_basic(code="sz.002030")
stockName = None
if (rs.error_code == '0') & rs.next():
    stockName = rs.get_row_data()[1]
print (stockName)
bs.logout()

login success!
达安基因
logout success!


In [7]:
customLogout()

logout success!


In [2]:

##
# 连接数据库，这里用的mongoDB
#
def connectDB():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["tinyspark"]
    return mydb

##
#  下载日K线数据
#  stockCode 股票代码
#  startDate 起始时间
#  endDate   结束时间
#
def downloadDailyStockKline(stockCode, startdate, enddate):
    return downloadPeriodStockKline("day", stockCode, startdate, enddate)

##
#  下载指定周期的K线数据
#  period 周期
#  stockCode 股票代码
#  startDate 起始时间
#  endDate   结束时间
#
def downloadPeriodStockKline(period, stockCode, startDate, endDate):
    frequency = chooseFrequency(period)
    customLogin()
    #查股票名字
    #返回示例数据
    #code	code_name	ipoDate	outDate	type	status
    #sh.600000	浦发银行	1999-11-10		1	1
    rs = bs.query_stock_basic(code=stockCode)
    stockName = None
    if (rs.error_code == '0') & rs.next():
        stockName = rs.get_row_data()[1]
    if stockName == None:
        raise RuntimeError("无此股票代码：", stockCode)
    
    #要查的字段，各周期有些许不同
    queryFields = []
    queryFields.append("date")
    if period.endswith("m"):
        queryFields.append("time")
    queryFields.append("open")
    queryFields.append("high")
    queryFields.append("low")
    queryFields.append("close")
    queryFields.append("volume")
    queryFields.append("amount")
    queryFields.append("adjustflag")
    if period == "day" or period == "week" or period == "month":
        queryFields.append("turn")
        queryFields.append("pctChg")
    if period == "day":
        queryFields.append("preclose")
        queryFields.append("tradestatus")
        queryFields.append("isST")
#     set_trace()
    queryFields = ",".join(queryFields)
#     queryFields = "date,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST"
    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
    rs = bs.query_history_k_data_plus(stockCode, queryFields,
        start_date=str(startDate), end_date=str(endDate),
        frequency=frequency, adjustflag="3")
    if rs.error_code != '0':
        raise RuntimeError("读" + stockCode + " 数据失败了")
    ##下载下来的数据，存数据库去
    writeKlineToDb(period, stockCode, stockName, rs)
    return True

##
#  写数据库
#
#
def writeKlineToDb(period, stockCode, stockName, resultSet):
    dataList = []
    while (resultSet.error_code == '0') & resultSet.next():
        # 获取一条记录，将记录合并在一起
#         data_list.append(rs.get_row_data())
        row = resultSet.get_row_data()
    
        kline = None
        recordDate = None
        
        
        if period == "day":
            kline = DayKline(stockCode, stockName)
        elif period == "week":
            kline = WeekKline(stockCode, stockName)
        elif period == "month":
            kline = MonthKline(stockCode, stockName)
        elif period == "5m":
            kline = FiveMinKline(stockCode, stockName)
        elif period == "30m":
            kline = ThirtyMinKline(stockCode, stockName)
        elif period == "60m":
            kline = SixtyMinKline(stockCode, stockName)
#         recordDate = datetime.strptime(row[0], "%Y-%m-%d")
        
        kline.openPrice = row[1]
        kline.highPrice = row[2]
        kline.lowPrice = row[3]
        kline.closePrice = row[4]
        kline.volume = row[5]
        kline.amount = row[6]
        kline.adjustflag = row[7]
        
        # 日K、月K、周K有专有属性
        if period == "day" or period == "week" or period == "month":
            recordDate = datetime.strptime(row[0], "%Y-%m-%d")
            kline.turn = row[8]
            kline.changePercent = row[9]
        else:
            recordDate = datetime.strptime(row[0], "%Y-%m-%dT%H:%M:%S.000Z")
        # 日K专有属性
        if period == "day":
            kline.preClosePrice = row[10]
            kline.tradeStatus = row[11]
            kline.isST = row[12]
        kline.date = recordDate
        
        dataList.append(kline.__dict__)

#     dataListJson = json.dumps(dataList, ensure_ascii=False) 
    mydb = connectDB()
    collection = mydb[chooseCollection(period)]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("数据为空")

##
#  选择不同的周期，调api用的
#
def chooseFrequency(period):
    frequency = {
        "day" : "d",
        "week" : "w",
        "month" : "m",
        "5m" : "5",
        "30m" : "30",
        "60m" : "60"
    }
    return frequency.get(period)

##
#  选择不同的Collection
#
def chooseCollection(period):
    periodCollection = {
        "day" : "Kline_Day",
        "week" : "Kline_Week",
        "month" : "Kline_Month",
        "5m" : "Kline_5m",
        "30m" : "Kline_30m",
        "60m" : "Kline_60m"
    }
    return periodCollection.get(period)
        
##
#  从数据库读取K线数据，转DataFrame
#  startDate、endDate在日线级别以上时，自动拼接成结束日23:59:59
#
def readStockKline(code, period, startDate, endDate):
    mydb = connectDB()
    cursor = None
    periodCollection = chooseCollection(period)
  
    if period == "day" or period == "week" or period == "month":
        startDate = datetime.strptime(startDate + "T00:00:00.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
        endDate = datetime.strptime(endDate + "T23:59:59.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
    
    
    cursor = mydb[periodCollection.get(period)].find({"code":code,"date":{"$gte":startDate, "$lte":endDate}})
    df =  pd.DataFrame(list(cursor))
    return df


def computeRSI(klineDataFrame):
    # 剔除停盘数据
    klineDataFrame = klineDataFrame[klineDataFrame['tradeStatus'] == '1']
    rsi_12days = ta.RSI(klineDataFrame['closePrice'],timeperiod=12)
    rsi_6days = ta.RSI(klineDataFrame['closePrice'],timeperiod=6)
    rsi_24days = ta.RSI(klineDataFrame['closePrice'],timeperiod=24)
    klineDataFrame['rsi_6days'] = rsi_6days
    klineDataFrame['rsi_12days'] = rsi_12days
    klineDataFrame['rsi_24days'] = rsi_24days
    ##添加参考线位置
    klineDataFrame['overBuy'] = rsiOverBuy
    klineDataFrame['overSell'] = rsiOverSell
    klineDataFrame['middle'] = rsiMiddle

    # RSI超卖和超买
    rsi_buy_position = klineDataFrame['rsi_6days'] > rsiOverBuy
    rsi_sell_position = klineDataFrame['rsi_6days'] < rsiOverSell
    klineDataFrame.loc[rsi_buy_position[(rsi_buy_position == True) & (rsi_buy_position.shift() == False)].index, '超买'] = '超买'
    klineDataFrame.loc[rsi_sell_position[(rsi_sell_position == True) & (rsi_sell_position.shift() == False)].index, '超卖'] = '超卖'
    return klineDataFrame

    

In [3]:
## 
# 获取指定日期的指数、股票数据
#
def allStocks(preferedDate):
    customLogin()
    if preferedDate == None:
        preferedDate = latestTradeDate()
    stock_rs = bs.query_all_stock(preferedDate)
    stock_df = stock_rs.get_data()
    return stock_df

def latestTradeDate():
    customLogin()
#     set_trace()
    rs = bs.query_trade_dates(start_date=date.today().replace(day=1), end_date = date.today())
    if rs.error_code != '0':
        raise RuntimeError("交易日api调用失败了:" + rs.error_code)
    tradeDates = []
    while (rs.error_code == '0') & rs.next():
        row = rs.get_row_data()
        if row[1] == "1":
            tradeDates.append(row[0])
#     set_trace()
    if len(tradeDates) == 0:
        raise RuntimeError("取不到最新的交易日")
    now = datetime.now()
    ## 因为baoStock的日K数据更新时间是 17：30， 所以如果在18点前启动，可能取不到当天数据，交易日向前推一天
    if (now.hour < 18 and len(tradeDates) > 1):
        return tradeDates[len(tradeDates) - 2]
    else:
        return tradeDates[len(tradeDates) - 1]

def downloadAllKlineDataOfSingleDay(date):
    customLogin()
    stock_df = allStocks(date)
    downloadedCount = 0
    
    for stockCode in stock_df["code"]:
        downloadDailyStockKline(stockCode, date, date)
        downloadedCount = downloadedCount + 1
        if downloadedCount % 100 == 0 and downloadedCount > 0:
            print ("process:", downloadedCount, " of ", len(stock_df) )
    bs.logout()

def downloadAllKlineDataOfPeriod(period, startDate):
    customLogin()
    if period.endswith("m"):
        startDate = datetime.strptime(startDate + "T00:00:00.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
        endDate = datetime.strptime(date.today() + "T23:59:59.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
    else:
        endDate = date.today()
    downloadedCount = 0
    failCount = 0
    stock_df = allStocks(None)
    for stockCode in stock_df["code"]:
        downloadedCount = downloadedCount + 1
        try:
            downloadPeriodStockKline(period, stockCode, startDate, endDate)
            
        except BaseException as e:
            failCount = failCount + 1
            print ("download " + stockCode + " error:" + str(e))
        
        if downloadedCount % 100 == 0 and downloadedCount > 0:
            print ("download process:", downloadedCount, " of ", len(stock_df) ," failed:", failCount)
    customLogout()

In [ ]:
# downloadAllKlineDataOfSingleDay("2019-09-19")
downloadAllKlineDataOfPeriod("day", "2017-01-01")
# allStocks(None)

login success!
download process: 100  of  4201  failed: 0
download process: 200  of  4201  failed: 0
download process: 300  of  4201  failed: 0
download process: 400  of  4201  failed: 0
download process: 500  of  4201  failed: 0
download process: 600  of  4201  failed: 0
download process: 700  of  4201  failed: 0
download process: 800  of  4201  failed: 0
download process: 900  of  4201  failed: 0
download process: 1000  of  4201  failed: 0
download process: 1100  of  4201  failed: 0
download process: 1200  of  4201  failed: 0
download process: 1300  of  4201  failed: 0
download process: 1400  of  4201  failed: 0
download process: 1500  of  4201  failed: 0
download process: 1600  of  4201  failed: 0
download process: 1700  of  4201  failed: 0
download process: 1800  of  4201  failed: 0
download process: 1900  of  4201  failed: 0
download process: 2000  of  4201  failed: 0
download process: 2100  of  4201  failed: 0


In [15]:
if __name__ == '__main__':
    code = "sz.002030"
    startDate = "2019-08-01"
    endDate = "2019-09-20"
    ## 下载数据
    result = downloadDailyStockKline(code, startDate, endDate)
    bs.logout()
#     df.to_csv("/Users/matt/Documents/pythonworkspace/dailyKline.csv", encoding='utf_8_sig')
    
#     df = pd.read_csv("/Users/matt/Documents/pythonworkspace/dailyKline.csv")
    
    # 从数据库读数据
#     df = readStockKline(code, "day", startDate, endDate)
    
#     ## 计算开始
#     rsi = computeRSI(df)
#     dateArray = []
#     for rsiDate in rsi["date"]:
#         rsiDate = rsiDate.strftime("%Y-%m-%d")
#         dateArray.append(rsiDate)
#     rsi["date"] = dateArray
#     df2 = rsi[['date','rsi_6days','rsi_12days','rsi_24days','overBuy','overSell','middle']]
#     df2.index = rsi['date']
#     plot = df2.plot(title='RSI', figsize=(40,10))
#     fig = plot.get_figure() 
#     fig.savefig('/Users/matt/Downloads/rsi.png')
#     rsi.to_csv("/Users/matt/Documents/pythonworkspace/rsi.csv",encoding='utf_8_sig')
    

login success!
login success!


BulkWriteError: batch op errors occurred